In [1]:
import os
import cv2
import numpy as np
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt

# Define constants
data_path = "/Users/yanthraa/Desktop/PROJECT/NEW DATASET/archive"
target_size = (224, 224)
target_count = 1000  # Desired number of images per class
output_path = "/Users/yanthraa/Desktop/PROJECT/NEW DATASET/Augmented_Dataset"  # Output directory

# Ensure output directories exist
os.makedirs(output_path, exist_ok=True)
for class_name in ['Negatives', 'Positives']:
    os.makedirs(os.path.join(output_path, class_name), exist_ok=True)

# Define augmentation sequence
augmenters = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal flip
    iaa.Affine(rotate=(-15, 15)),  # Rotate between -15 and 15 degrees
    iaa.GaussianBlur(sigma=(0.0, 1.0)),  # Apply Gaussian blur
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add noise
    iaa.Multiply((0.8, 1.2))  # Brightness adjustments
])

# Load and preprocess data
images = []
labels = []

for class_name in ['Negatives', 'Positives']:
    class_path = os.path.join(data_path, class_name)
    class_images = []
    label = 0 if class_name == 'Negatives' else 1
    
    # Load original images
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, target_size)
        img = img.astype(np.float32) / 255.0
        class_images.append(img)
    
    # Augment images until target count is reached
    while len(class_images) < target_count:
        augmented_images = augmenters(images=class_images[:min(len(class_images), target_count - len(class_images))])
        class_images.extend(augmented_images)
    
    class_images = class_images[:target_count]  # Trim excess
    images.extend(class_images)
    labels.extend([label] * target_count)
    
    # Save augmented images
    for idx, img in enumerate(class_images):
        save_path = os.path.join(output_path, class_name, f"aug_{idx}.png")
        cv2.imwrite(save_path, (img * 255).astype(np.uint8))

# Convert to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Print class proportions
print("Class Proportions:")
print("Negatives:", np.sum(labels == 0), "Positives:", np.sum(labels == 1))


Class Proportions:
Negatives: 1000 Positives: 1000


In [2]:
from sklearn.model_selection import train_test_split

# Separate data for Negatives and Positives classes
Negatives_images = images[labels == 0][:500]
Positives_images = images[labels == 1][:500]
Negatives_labels = labels[labels == 0][:500]
Positives_labels = labels[labels == 1][:500]

# Concatenate the data back together
balanced_images = np.concatenate([Negatives_images, Positives_images])
balanced_labels = np.concatenate([Negatives_labels, Positives_labels])


# Split the data into training and testing sets
X_train_full, X_test, y_train_full, y_test = train_test_split(balanced_images, balanced_labels, test_size=0.2, random_state=42)

# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.1, random_state=42)

#print the balanced data
print("Balanced Images:",balanced_images.shape)
print("Balanced labels:",balanced_labels.shape)

#print the data seperately of each class
print("Negatives_images:",Negatives_images.shape)
print("Positives_images:",Positives_images.shape)


# Print the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


Balanced Images: (1000, 224, 224)
Balanced labels: (1000,)
Negatives_images: (500, 224, 224)
Positives_images: (500, 224, 224)
X_train shape: (720, 224, 224)
X_test shape: (200, 224, 224)
y_train shape: (720,)
y_test shape: (200,)


In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score
# Add these new imports:
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score
import pandas as pd
import time
from datetime import datetime


In [6]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import (
    f1_score, confusion_matrix, log_loss, matthews_corrcoef,
    balanced_accuracy_score, cohen_kappa_score, roc_auc_score
)
import pandas as pd
import time
from datetime import datetime
import gc
import os

# Enable memory growth to prevent GPU memory allocation issues
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("GPU memory growth enabled")

# Add channel dimension to the input data - simplified
def preprocess_data(X):
    if len(X.shape) == 3:
        X = np.expand_dims(X, axis=-1)
        X = np.repeat(X, 3, axis=-1)
    return X

# Lightweight WindowAttention optimized for CPU
class WindowAttention(layers.Layer):
    def __init__(self, window_size, num_heads, dropout_rate=0.1, **kwargs):
        super(WindowAttention, self).__init__(**kwargs)
        self.window_size = window_size
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

    def build(self, input_shape):
        # Significantly smaller key dimension for CPU
        key_dim = max(16, input_shape[-1] // (self.num_heads * 2))
        
        # For CPU, consider using a simpler attention mechanism
        self.mha = layers.MultiHeadAttention(
            num_heads=self.num_heads,
            key_dim=key_dim,
            dropout=self.dropout_rate
        )
        
        # Cache input shape for faster processing
        self.input_height = input_shape[1]
        self.input_width = input_shape[2]
        self.channels = input_shape[3]
        
        # Add direct projection option as alternative to full attention
        self.use_lightweight = (self.window_size > 4)  # Use lightweight for larger windows
        if self.use_lightweight:
            # Create a lightweight alternative that's faster on CPU
            self.conv_proj = tf.keras.Sequential([
                layers.Conv2D(self.channels, kernel_size=3, padding='same'),
                layers.BatchNormalization(),
                layers.Activation('relu')
            ])
            
        self.built = True

    def call(self, inputs, training=False):
        # For very large windows on CPU, use the lightweight path
        if self.use_lightweight and not training:
            return self.conv_proj(inputs)
            
        B = tf.shape(inputs)[0]
        H, W, C = self.input_height, self.input_width, self.channels
        
        # Pre-calculate padding to avoid conditional logic
        pad_h = (self.window_size - (H % self.window_size)) % self.window_size
        pad_w = (self.window_size - (W % self.window_size)) % self.window_size
        
        # Apply padding if needed
        if pad_h > 0 or pad_w > 0:
            x = tf.pad(inputs, [[0, 0], [0, pad_h], [0, pad_w], [0, 0]])
            padded_H, padded_W = H + pad_h, W + pad_w
        else:
            x = inputs
            padded_H, padded_W = H, W
        
        # Use static window counts for better CPU performance
        num_h_windows = padded_H // self.window_size
        num_w_windows = padded_W // self.window_size
        
        # Use a more direct reshape flow for CPU
        x = tf.reshape(x, [B, num_h_windows, self.window_size, num_w_windows, self.window_size, C])
        x = tf.transpose(x, [0, 1, 3, 2, 4, 5])
        x = tf.reshape(x, [-1, self.window_size * self.window_size, C])
        
        # Apply attention
        attention_output = self.mha(x, x, training=training)
        
        # Reshape back directly
        x = tf.reshape(attention_output, [B, num_h_windows, num_w_windows, self.window_size, self.window_size, C])
        x = tf.transpose(x, [0, 1, 3, 2, 4, 5])
        x = tf.reshape(x, [B, padded_H, padded_W, C])
        
        # Remove padding if needed
        if pad_h > 0 or pad_w > 0:
            return x[:, :H, :W, :]
        else:
            return x

# Simplified MLP block
class MLPBlock(layers.Layer):
    def __init__(self, hidden_units, dropout_rate=0.1, **kwargs):
        super(MLPBlock, self).__init__(**kwargs)
        self.mlp = tf.keras.Sequential([
            layers.Dense(units, activation='gelu') 
            for units in hidden_units
        ])
        self.dropout = layers.Dropout(dropout_rate)
        
    def call(self, inputs, training=False):
        x = self.mlp(inputs)
        return self.dropout(x, training=training)

# Optimized MaxViT block
class MaxViTBlock(layers.Layer):
    def __init__(self, num_heads=8, mlp_units=[128, 64], window_size=7, dropout_rate=0.1, **kwargs):
        super(MaxViTBlock, self).__init__(**kwargs)
        
        # Initialize layers
        self.window_attention = WindowAttention(
            window_size=window_size, 
            num_heads=num_heads, 
            dropout_rate=dropout_rate
        )
        self.mlp = MLPBlock(
            hidden_units=mlp_units, 
            dropout_rate=dropout_rate
        )
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training=False):
        # Attention branch
        x_norm = self.norm1(inputs)
        attention_output = self.window_attention(x_norm, training=training)
        x = inputs + attention_output
        
        # MLP branch
        x_norm2 = self.norm2(x)
        mlp_output = self.mlp(x_norm2, training=training)
        x = x + mlp_output

        return x

def create_maxvit_model(input_shape=(28, 28, 3), num_classes=1, depth=2):
    """
    Create a highly efficient MaxViT model for CPU-only environments
    """
    inputs = Input(shape=input_shape)
    
    # Initial embedding with more aggressive downsampling for CPU
    x = layers.Conv2D(16, kernel_size=5, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)  # Using relu instead of gelu for CPU efficiency
    
    # Progressive feature dimension - much smaller for CPU
    features = [16, 32, 64, 128][:depth]  # Reduced feature dimensions
    
    # Calculate static window sizes ahead of time
    h, w = input_shape[0] // 2, input_shape[1] // 2
    
    # MaxViT blocks - reduced complexity for CPU
    for i, dim in enumerate(features):
        # Feature dimension adjustment
        if i > 0:
            x = layers.Conv2D(dim, kernel_size=1, padding="same")(x)
            x = layers.BatchNormalization()(x)
        
        # Use a smaller fixed window size for CPU
        window_size = max(2, min(5, h // 2))  # Reduced from max 7 to max 5
        
        # MaxViT block with fewer heads for CPU
        x = MaxViTBlock(
            num_heads=max(1, min(4, dim // 16)),  # Reduced max heads from 8 to 4
            mlp_units=[dim, dim],  # Simpler MLP structure
            window_size=window_size,
            dropout_rate=0.1
        )(x)
        
        # More aggressive downsampling for CPU
        if i < len(features) - 1:
            x = layers.AveragePooling2D(pool_size=2)(x)  # Pool instead of Conv2D for CPU
            h, w = h // 2, w // 2
    
    # Global pooling and simpler classification head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(32, activation="relu")(x)  # Reduced from 64 to 32
    x = layers.Dropout(0.2)(x)
    
    # Output layer
    if num_classes == 1:
        outputs = layers.Dense(1, activation="sigmoid")(x)
    else:
        outputs = layers.Dense(num_classes, activation="softmax")(x)
    
    return Model(inputs, outputs)

def calculate_metrics(y_true, y_pred, y_pred_proba=None):
    """Calculate all required metrics"""
    
    # Convert predictions to binary
    if len(y_pred.shape) > 1 and y_pred.shape[1] > 1:
        y_pred_binary = np.argmax(y_pred, axis=1)
    else:
        y_pred_binary = (y_pred > 0.5).astype(int)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_binary).ravel()
    
    # Basic metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Calculate additional metrics
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    mcc = matthews_corrcoef(y_true, y_pred_binary)
    
    # Calculate log loss if probabilities provided
    if y_pred_proba is not None:
        log_loss_val = log_loss(y_true, y_pred_proba)
    else:
        y_pred_clipped = np.clip(y_pred, 1e-15, 1 - 1e-15)
        log_loss_val = -np.mean(y_true * np.log(y_pred_clipped) + 
                        (1 - y_true) * np.log(1 - y_pred_clipped))
    
    # Additional metrics
    g_mean = np.sqrt(recall * specificity) if (recall * specificity) > 0 else 0
    youdens_j = recall + specificity - 1
    balanced_acc = balanced_accuracy_score(y_true, y_pred_binary)
    kappa = cohen_kappa_score(y_true, y_pred_binary)
    
    # AUC-ROC score
    try:
        if y_pred_proba is not None:
            auc_roc = roc_auc_score(y_true, y_pred_proba)
        else:
            auc_roc = roc_auc_score(y_true, y_pred)
    except:
        auc_roc = 0.5
    
    return {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1_Score': f1,
        'AUC_ROC': auc_roc,
        'TP': tp,
        'TN': tn,
        'FP': fp,
        'FN': fn,
        'Specificity': specificity,
        'MCC': mcc,
        'Log_Loss': log_loss_val,
        'G_Mean': g_mean,
        'Youdens_J': youdens_j,
        'Balanced_Accuracy': balanced_acc,
        'Cohens_Kappa': kappa
    }

def perform_kfold_cv(X, y, X_test=None, y_test=None, k_folds=[3], epochs=20):
    """
    Perform k-fold cross-validation with the MaxViT model, optimized for CPU-only environments
    """
    # Reduce input dimensions for CPU by downsampling large inputs 
    def downsample_if_needed(X):
        # If inputs are large, downsample them for CPU efficiency
        if X.shape[1] > 64 or X.shape[2] > 64:
            from skimage.transform import resize
            factor = max(1, min(X.shape[1], X.shape[2]) // 64)
            target_shape = (X.shape[0], X.shape[1]//factor, X.shape[2]//factor)
            X_small = np.zeros(target_shape + (X.shape[3],) if len(X.shape) > 3 else target_shape)
            
            for i in range(X.shape[0]):
                if len(X.shape) > 3:
                    X_small[i] = resize(X[i], (target_shape[1], target_shape[2], X.shape[3]))
                else:
                    X_small[i] = resize(X[i], (target_shape[1], target_shape[2]))
            
            print(f"Downsampled data from {X.shape[1]}x{X.shape[2]} to {X_small.shape[1]}x{X_small.shape[2]}")
            return X_small
        return X
    
    # Preprocess and downsample the data for CPU
    X = downsample_if_needed(X)
    X = preprocess_data(X)
    if X_test is not None:
        X_test = downsample_if_needed(X_test)
        X_test = preprocess_data(X_test)
    
    # Results file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f'outputresults/maxvit_results_{timestamp}111.csv'
    all_results = []
    
    # More aggressive early stopping for CPU
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=2,  # Very low patience for CPU
            restore_best_weights=True,
            min_delta=0.02  # Higher delta for faster termination
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.1,  # Very aggressive reduction
            patience=1,  # Minimal patience
            min_lr=1e-4  # Higher min_lr to avoid too many steps
        )
    ]
    
    # Set thread optimization for CPU
    tf.config.threading.set_inter_op_parallelism_threads(2)  # Reduce thread contention
    tf.config.threading.set_intra_op_parallelism_threads(4)  # Set reasonable thread count
    
    # Run for each k-fold setting
    for k in k_folds:
        print(f"\n=== Starting {k}-fold cross-validation ===")
        
        # Initialize K-fold
        kfold = KFold(n_splits=k, shuffle=True, random_state=42)
        
        for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
            print(f'\nFold {fold + 1}/{k}')
            
            # Split data - use smaller validation set for CPU
            train_size = min(len(train_idx), 1000)  # Limit training set size
            val_size = min(len(val_idx), 200)  # Limit validation set size
            
            # Subsample data for faster CPU processing
            train_idx = train_idx[:train_size]
            val_idx = val_idx[:val_size]
            
            X_train_fold, X_val_fold = X[train_idx], X[val_idx]
            y_train_fold, y_val_fold = y[train_idx], y[val_idx]
            
            print(f"Training data shape: {X_train_fold.shape} (limited for CPU)")
            
            # Get input shape from actual data
            input_shape = X_train_fold.shape[1:]
            
            # Create model with minimal depth for CPU
            model = create_maxvit_model(input_shape=input_shape, depth=2)  # Ultra-light depth for CPU
            
            # Use simpler optimizer for CPU
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)  # Higher learning rate
                
            model.compile(
                optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy']  # Minimal metrics for CPU
            )
            
            # Train model with optimal CPU batch size
            start_time = time.time()
            batch_size = 32  # Smaller batch size for CPU memory efficiency
            history = model.fit(
                X_train_fold, y_train_fold,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(X_val_fold, y_val_fold),
                callbacks=callbacks,
                verbose=1
            )
            training_time = time.time() - start_time
            
            # Evaluate on validation set
            val_results = model.evaluate(X_val_fold, y_val_fold, verbose=0)
            val_pred = model.predict(X_val_fold, verbose=0, batch_size=batch_size)
            val_metrics = calculate_metrics(y_val_fold, val_pred)
            
            # Evaluate on test set if provided
            if X_test is not None and y_test is not None:
                test_results = model.evaluate(X_test, y_test, verbose=0)
                test_pred = model.predict(X_test, verbose=0, batch_size=batch_size)
                test_metrics = calculate_metrics(y_test, test_pred)
                eval_metrics = test_metrics
                eval_loss = test_results[0]
            else:
                eval_metrics = val_metrics
                eval_loss = val_results[0]
            
            # Store metrics
            fold_metrics = {
                'Fold': fold + 1,
                'K': k,
                'Test_Accuracy': eval_metrics['Accuracy'],
                'Train_Accuracy': history.history['accuracy'][-1],
                'Precision': eval_metrics['Precision'],
                'Recall': eval_metrics['Recall'],
                'AUC_ROC': eval_metrics['AUC_ROC'],
                'F1_Score': eval_metrics['F1_Score'],
                'Training_Loss': history.history['loss'][-1],
                'Testing_Loss': eval_loss,
                'Time_Taken': training_time,
                'Epochs_Run': len(history.history['loss']),
                'TP': eval_metrics['TP'],
                'TN': eval_metrics['TN'],
                'FP': eval_metrics['FP'],
                'FN': eval_metrics['FN'],
                'Specificity': eval_metrics['Specificity'],
                'MCC': eval_metrics['MCC'],
                'Log_Loss': eval_metrics['Log_Loss'],
                'G_Mean': eval_metrics['G_Mean'],
                'Youdens_J': eval_metrics['Youdens_J'],
                'Balanced_Accuracy': eval_metrics['Balanced_Accuracy'],
                'Cohens_Kappa': eval_metrics['Cohens_Kappa']
            }
            
            # Add to results list
            all_results.append(fold_metrics)
            
            # Print current fold results
            print(f"\nFold {fold + 1} Results:")
            for metric, value in fold_metrics.items():
                if isinstance(value, (int, float)):
                    print(f"{metric}: {value:.4f}")
            
            # Clear memory
            del model
            del history
            tf.keras.backend.clear_session()
            gc.collect()
    
    # Convert to DataFrame and save to CSV
    results_df = pd.DataFrame(all_results)
    results_df.to_csv(results_file, index=False)
    
    # Calculate and print summary statistics
    print("\n=== Summary Statistics ===")
    for k in k_folds:
        k_results = results_df[results_df['K'] == k]
        print(f"\nK={k} Cross-Validation Results:")
        print(f"Average Test Accuracy: {k_results['Test_Accuracy'].mean():.4f} ± {k_results['Test_Accuracy'].std():.4f}")
        print(f"Average F1 Score: {k_results['F1_Score'].mean():.4f} ± {k_results['F1_Score'].std():.4f}")
        print(f"Average AUC-ROC: {k_results['AUC_ROC'].mean():.4f} ± {k_results['AUC_ROC'].std():.4f}")
        print(f"Average Training Time: {k_results['Time_Taken'].mean():.2f} seconds")
        print(f"Average Epochs Run: {k_results['Epochs_Run'].mean():.1f}")
    
    return results_df

# Usage example
if __name__ == "__main__":
    # Uncomment and run when needed
    
    # For 5-fold and 10-fold validation with test set evaluation
    results = perform_kfold_cv(
        X=X_train_full, 
        y=y_train_full,
        X_test=X_test,
        y_test=y_test,
        k_folds=[5,10,15,20],  # Changed from [5, 10] to just [5] for faster execution
        epochs=50     # Reduced from 50 to 30
    )
    

Downsampled data from 224x224 to 74x74
Downsampled data from 224x224 to 74x74

=== Starting 5-fold cross-validation ===

Fold 1/5
Training data shape: (640, 74, 74, 3) (limited for CPU)
Epoch 1/50


/opt/anaconda3/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['max_vi_t_block/window_attention/sequential_1/conv2d_1/kernel', 'max_vi_t_block/window_attention/sequential_1/conv2d_1/bias', 'max_vi_t_block/window_attention/sequential_1/batch_normalization_1/gamma', 'max_vi_t_block/window_attention/sequential_1/batch_normalization_1/beta', 'max_vi_t_block_1/window_attention_1/sequential_3/conv2d_3/kernel', 'max_vi_t_block_1/window_attention_1/sequential_3/conv2d_3/bias', 'max_vi_t_block_1/window_attention_1/sequential_3/batch_normalization_3/gamma', 'max_vi_t_block_1/window_attention_1/sequential_3/batch_normalization_3/beta'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.6730 - loss: 0.6545 - val_accuracy: 0.5188 - val_loss: 0.7396 - learning_rate: 0.0050
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 102ms/step - accuracy: 0.7422 - loss: 0.5864 - val_accuracy: 0.5188 - val_loss: 0.7470 - learning_rate: 0.0050
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step - accuracy: 0.7636 - loss: 0.5390 - val_accuracy: 0.5188 - val_loss: 0.7194 - learning_rate: 5.0000e-04
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - accuracy: 0.7407 - loss: 0.5457 - val_accuracy: 0.5188 - val_loss: 0.7427 - learning_rate: 5.0000e-04
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 123ms/step - accuracy: 0.7390 - loss: 0.5342 - val_accuracy: 0.5188 - val_loss: 0.7545 - learning_rate: 1.0000e-04

Fold 1 Results:
Fold: 1.0000
K: 5.0000
Test_Accuracy: 0.4800
Train_Accuracy: 0.7563
Precision: 0.0000
Recall: 0.0000
AUC_ROC: 0.7945
F1_Score: 0.0000
Training_Loss: 0.5164
Testing_Loss: 0.7593
Time_Taken: 15.2012
Epochs_Run: 5.0000
Specif

/opt/anaconda3/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['max_vi_t_block/window_attention/sequential_1/conv2d_1/kernel', 'max_vi_t_block/window_attention/sequential_1/conv2d_1/bias', 'max_vi_t_block/window_attention/sequential_1/batch_normalization_1/gamma', 'max_vi_t_block/window_attention/sequential_1/batch_normalization_1/beta', 'max_vi_t_block_1/window_attention_1/sequential_3/conv2d_3/kernel', 'max_vi_t_block_1/window_attention_1/sequential_3/conv2d_3/bias', 'max_vi_t_block_1/window_attention_1/sequential_3/batch_normalization_3/gamma', 'max_vi_t_block_1/window_attention_1/sequential_3/batch_normalization_3/beta'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.6385 - loss: 0.6705 - val_accuracy: 0.5188 - val_loss: 0.6873 - learning_rate: 0.0050
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 109ms/step - accuracy: 0.7673 - loss: 0.5563 - val_accuracy: 0.5063 - val_loss: 0.7048 - learning_rate: 0.0050
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.7211 - loss: 0.6057 - val_accuracy: 0.5063 - val_loss: 0.6935 - learning_rate: 5.0000e-04

Fold 2 Results:
Fold: 2.0000
K: 5.0000
Test_Accuracy: 0.5500
Train_Accuracy: 0.7594
Precision: 0.5365
Recall: 0.9904
AUC_ROC: 0.7379
F1_Score: 0.6959
Training_Loss: 0.5544
Testing_Loss: 0.6745
Time_Taken: 10.6485
Epochs_Run: 3.0000
Specificity: 0.0729
MCC: 0.1614
Log_Loss: 0.6958
G_Mean: 0.2687
Youdens_J: 0.0633
Balanced_Accuracy: 0.5317
Cohens_Kappa: 0.0656

Fold 3/5
Training data shape: (640, 74, 74, 3) (limited for CPU)
Epoch 1/50


...

24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 112ms/step - accuracy: 0.7133 - loss: 0.6093 - val_accuracy: 0.5000 - val_loss: 0.8259 - learning_rate: 0.0050
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.7581 - loss: 0.5668 - val_accuracy: 0.5000 - val_loss: 0.9201 - learning_rate: 0.0050
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.7575 - loss: 0.5509 - val_accuracy: 0.5000 - val_loss: 0.9788 - learning_rate: 5.0000e-04

Fold 18 Results:
Fold: 18.0000
K: 20.0000
Test_Accuracy: 0.4800
Train_Accuracy: 0.7566
Precision: 0.0000
Recall: 0.0000
AUC_ROC: 0.5184
F1_Score: 0.0000
Training_Loss: 0.5468
Testing_Loss: 0.8537
Time_Taken: 11.6699
Epochs_Run: 3.0000
Specificity: 1.0000
MCC: 0.0000
Log_Loss: 0.8585
G_Mean: 0.0000
Youdens_J: 0.0000
Balanced_Accuracy: 0.5000
Cohens_Kappa: 0.0000

Fold 19/20
Training data shape: (760, 74, 74, 3) (limited for CPU)
Epoch 1/50


/opt/anaconda3/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['max_vi_t_block/window_attention/sequential_1/conv2d_1/kernel', 'max_vi_t_block/window_attention/sequential_1/conv2d_1/bias', 'max_vi_t_block/window_attention/sequential_1/batch_normalization_1/gamma', 'max_vi_t_block/window_attention/sequential_1/batch_normalization_1/beta', 'max_vi_t_block_1/window_attention_1/sequential_3/conv2d_3/kernel', 'max_vi_t_block_1/window_attention_1/sequential_3/conv2d_3/bias', 'max_vi_t_block_1/window_attention_1/sequential_3/batch_normalization_3/gamma', 'max_vi_t_block_1/window_attention_1/sequential_3/batch_normalization_3/beta'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 118ms/step - accuracy: 0.7134 - loss: 0.5932 - val_accuracy: 0.2750 - val_loss: 1.2593 - learning_rate: 0.0050
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.7016 - loss: 0.6111 - val_accuracy: 0.2750 - val_loss: 1.1276 - learning_rate: 0.0050
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.7508 - loss: 0.5675 - val_accuracy: 0.2750 - val_loss: 0.9860 - learning_rate: 0.0050
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.7770 - loss: 0.5257 - val_accuracy: 0.2750 - val_loss: 0.8416 - learning_rate: 0.0050
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.7522 - loss: 0.5281 - val_accuracy: 0.2750 - val_loss: 0.7170 - learning_rate: 0.0050
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.7443 - loss: 0.5469 - val_accuracy: 0.2750 - val_loss: 0.9039 - learning_rate: 0.0050
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 112ms/step - accuracy: 0.7373 - loss: 0.5598 - val_accuracy: 0.

/opt/anaconda3/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['max_vi_t_block/window_attention/sequential_1/conv2d_1/kernel', 'max_vi_t_block/window_attention/sequential_1/conv2d_1/bias', 'max_vi_t_block/window_attention/sequential_1/batch_normalization_1/gamma', 'max_vi_t_block/window_attention/sequential_1/batch_normalization_1/beta', 'max_vi_t_block_1/window_attention_1/sequential_3/conv2d_3/kernel', 'max_vi_t_block_1/window_attention_1/sequential_3/conv2d_3/bias', 'max_vi_t_block_1/window_attention_1/sequential_3/batch_normalization_3/gamma', 'max_vi_t_block_1/window_attention_1/sequential_3/batch_normalization_3/beta'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 114ms/step - accuracy: 0.6610 - loss: 0.6353 - val_accuracy: 0.5500 - val_loss: 0.6628 - learning_rate: 0.0050
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.7527 - loss: 0.5754 - val_accuracy: 0.7500 - val_loss: 0.6152 - learning_rate: 0.0050
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 110ms/step - accuracy: 0.7589 - loss: 0.5536 - val_accuracy: 0.7750 - val_loss: 0.6056 - learning_rate: 0.0050
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.7609 - loss: 0.5415 - val_accuracy: 0.6000 - val_loss: 0.6198 - learning_rate: 0.0050

Fold 20 Results:
Fold: 20.0000
K: 20.0000
Test_Accuracy: 0.7150
Train_Accuracy: 0.7618
Precision: 0.6767
Recall: 0.8654
AUC_ROC: 0.7458
F1_Score: 0.7595
Training_Loss: 0.5429
Testing_Loss: 0.6143
Time_Taken: 14.3171
Epochs_Run: 4.0000
Specificity: 0.5521
MCC: 0.4419
Log_Loss: 0.7284
G_Mean: 0.6912
Youdens_J: 0.4175
Balanced_Accuracy: 0.7087
Cohens_Kappa: 0.4224

=== Summary Statistics ===

K=5 Cross-V

In [7]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import (
    f1_score, confusion_matrix, roc_auc_score,
    matthews_corrcoef, balanced_accuracy_score, cohen_kappa_score
)
import pandas as pd
import time
from datetime import datetime
import os
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Notice we don't import TensorFlow at all

print("Using scikit-learn for CPU compatibility")

def preprocess_data(X):
    # Downsample large images for CPU efficiency
    if X.shape[1] > 112 or X.shape[2] > 112:  # More aggressive downsampling
        from skimage.transform import resize
        factor = max(1, min(X.shape[1], X.shape[2]) // 112)
        new_shape = (X.shape[0], X.shape[1]//factor, X.shape[2]//factor)
        X_small = np.zeros(new_shape + (X.shape[3],) if len(X.shape) > 3 else new_shape)
        
        for i in range(X.shape[0]):
            if len(X.shape) > 3:
                X_small[i] = resize(X[i], (new_shape[1], new_shape[2], X.shape[3]))
            else:
                X_small[i] = resize(X[i], (new_shape[1], new_shape[2]))
        
        print(f"Downsampled data from {X.shape[1]}x{X.shape[2]} to {X_small.shape[1]}x{X_small.shape[2]}")
        X = X_small
    
    # Add channel dimension if it doesn't exist
    if len(X.shape) == 3:
        X = np.expand_dims(X, axis=-1)
        X = np.repeat(X, 3, axis=-1)
    return X

def extract_features(X):
    """Extract simple features from images for use in sklearn models"""
    # Reshape from (N, H, W, C) to (N, H*W*C)
    # or simply flatten each image
    n_samples = X.shape[0]
    
    # Simple feature extraction
    features = []
    
    for i in range(n_samples):
        img = X[i]
        
        # Average pooling to reduce dimensionality
        h_pools = img.shape[0] // 8
        w_pools = img.shape[1] // 8
        pooled = np.zeros((8, 8, img.shape[2]))
        
        for h in range(8):
            for w in range(8):
                h_start = h * h_pools
                h_end = (h + 1) * h_pools
                w_start = w * w_pools
                w_end = (w + 1) * w_pools
                pooled[h, w] = np.mean(img[h_start:h_end, w_start:w_end], axis=(0, 1))
        
        # Extract more features
        # 1. Mean of each channel
        channel_means = np.mean(img, axis=(0, 1))
        
        # 2. Standard deviation of each channel
        channel_stds = np.std(img, axis=(0, 1))
        
        # 3. Histogram features (simplified)
        hist_features = []
        for c in range(img.shape[2]):
            hist, _ = np.histogram(img[:, :, c], bins=10, range=(0, 1))
            hist_features.extend(hist / np.sum(hist))  # Normalize
        
        # Combine features
        feature_vector = np.concatenate([
            pooled.flatten(),
            channel_means,
            channel_stds,
            np.array(hist_features)
        ])
        
        features.append(feature_vector)
    
    return np.array(features)

def perform_kfold_cv(X, y, k_folds=[5, 10], max_iter=100):
    """scikit-learn based k-fold cross-validation with multiple k values"""
    # Preprocess and downsample
    X = preprocess_data(X)
    print(f"Data shape after preprocessing: {X.shape}")
    
    # Extract features for scikit-learn model
    print("Extracting features...")
    X_features = extract_features(X)
    print(f"Feature shape: {X_features.shape}")
    
    # Setup results storage
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f'outputresults/mobilesam_results_{timestamp}111.csv'
    all_results = []
    
    # Run for each k-fold setting
    for k in k_folds:
        print(f"\n=== Starting {k}-fold cross-validation ===")
        
        # Setup k-fold for this k value
        kfold = KFold(n_splits=k, shuffle=True, random_state=42)
        
        for fold, (train_idx, val_idx) in enumerate(kfold.split(X_features)):
            print(f'\nFold {fold + 1}/{k}')
            
            # Limit training set size for CPU efficiency
            max_train_size = min(len(train_idx), 800)
            max_val_size = min(len(val_idx), 200)
            train_idx = train_idx[:max_train_size]
            val_idx = val_idx[:max_val_size]
            
            # Get data for this fold
            X_train_fold = X_features[train_idx].astype('float32')
            X_val_fold = X_features[val_idx].astype('float32')
            y_train_fold = y[train_idx].astype('float32')
            y_val_fold = y[val_idx].astype('float32')
            
            print(f"Training data shape: {X_train_fold.shape} (limited for CPU)")
            print(f"Validation data shape: {X_val_fold.shape}")
            
            # Scale features
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train_fold)
            X_val_scaled = scaler.transform(X_val_fold)
            
            # Create model - scikit-learn MLP classifier
            model = MLPClassifier(
                hidden_layer_sizes=(128, 64),  # Two hidden layers
                activation='relu',
                solver='adam',
                alpha=0.0001,  # L2 penalty
                batch_size=16,
                learning_rate='adaptive',
                learning_rate_init=0.001,
                max_iter=max_iter,  # Number of epochs
                early_stopping=True,  # Use early stopping
                validation_fraction=0.1,  # 10% of training data for early stopping
                n_iter_no_change=5,  # Patience for early stopping
                verbose=True,
                random_state=42
            )
            
            print(f"Training scikit-learn MLP model...")
            
            # Train the model
            start_time = time.time()
            model.fit(X_train_scaled, y_train_fold.ravel())
            training_time = time.time() - start_time
            
            print(f"Training completed in {training_time:.2f} seconds")
            
            # Get predictions
            val_pred_proba = model.predict_proba(X_val_scaled)[:, 1]  # Get probability of class 1
            val_pred_binary = model.predict(X_val_scaled)
            
            # Calculate metrics
            tn, fp, fn, tp = confusion_matrix(y_val_fold, val_pred_binary).ravel()
            
            # Basic metrics
            accuracy = (tp + tn) / (tp + tn + fp + fn)
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            # Additional metrics
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
            mcc = matthews_corrcoef(y_val_fold, val_pred_binary)
            
            # Calculate log loss
            y_pred_clipped = np.clip(val_pred_proba, 1e-15, 1 - 1e-15)
            log_loss_val = -np.mean(y_val_fold * np.log(y_pred_clipped) + 
                            (1 - y_val_fold) * np.log(1 - y_pred_clipped))
            
            # Other metrics
            g_mean = np.sqrt(recall * specificity) if (recall * specificity) > 0 else 0
            youdens_j = recall + specificity - 1
            balanced_acc = balanced_accuracy_score(y_val_fold, val_pred_binary)
            kappa = cohen_kappa_score(y_val_fold, val_pred_binary)
            
            # Calculate AUC if possible
            try:
                auc_roc = roc_auc_score(y_val_fold, val_pred_proba)
            except:
                auc_roc = 0.5
            
            # Store metrics
            fold_metrics = {
                'Fold': fold + 1,
                'K': k,
                'Test_Accuracy': accuracy,
                'Train_Accuracy': model.score(X_train_scaled, y_train_fold),
                'Precision': precision,
                'Recall': recall,
                'AUC_ROC': auc_roc,
                'F1_Score': f1,
                'Training_Loss': model.loss_,
                'Testing_Loss': log_loss_val,
                'Time_Taken': training_time,
                'Epochs_Run': model.n_iter_,
                'TP': tp,
                'TN': tn,
                'FP': fp,
                'FN': fn,
                'Specificity': specificity,
                'MCC': mcc,
                'Log_Loss': log_loss_val,
                'G_Mean': g_mean,
                'Youdens_J': youdens_j,
                'Balanced_Accuracy': balanced_acc,
                'Cohens_Kappa': kappa
            }
            
            all_results.append(fold_metrics)
            
            # Print current fold results
            print(f"\nFold {fold + 1} Results:")
            for metric, value in fold_metrics.items():
                if isinstance(value, (int, float)):
                    print(f"{metric}: {value:.4f}")
            
            # Try to save the model
            try:
                from joblib import dump
                dump(model, f'sklearn_model_fold_{fold+1}.joblib')
                print(f"Model saved to sklearn_model_fold_{fold+1}.joblib")
            except:
                print("Could not save model")
        
    # Save all results
    results_df = pd.DataFrame(all_results)
    results_df.to_csv(results_file, index=False)
    
    # Calculate and print summary statistics for each k value
    print("\n=== Summary Statistics ===")
    for k in k_folds:
        k_results = results_df[results_df['K'] == k]
        print(f"\nK={k} Cross-Validation Results:")
        print(f"Average Test Accuracy: {k_results['Test_Accuracy'].mean():.4f} ± {k_results['Test_Accuracy'].std():.4f}")
        print(f"Average F1 Score: {k_results['F1_Score'].mean():.4f} ± {k_results['F1_Score'].std():.4f}")
        print(f"Average AUC-ROC: {k_results['AUC_ROC'].mean():.4f} ± {k_results['AUC_ROC'].std():.4f}")
        print(f"Average Training Time: {k_results['Time_Taken'].mean():.2f} seconds")
        print(f"Average Epochs Run: {k_results['Epochs_Run'].mean():.1f}")
    
    return results_df

# Example usage
if __name__ == "__main__":
    # Import time for benchmarking
    import time
    start_time = time.time()
    
    # Run with requested fold parameters
    results = perform_kfold_cv(
        X=X_train_full, 
        y=y_train_full,
        k_folds=[5, 10,15,20],  # Run both 5-fold and 10-fold CV
        max_iter=100      # Number of epochs/iterations
    )
    
    print(f"\nTotal execution time: {time.time() - start_time:.2f} seconds")

Using scikit-learn for CPU compatibility
Downsampled data from 224x224 to 112x112
Data shape after preprocessing: (800, 112, 112, 3)
Extracting features...
Feature shape: (800, 228)

=== Starting 5-fold cross-validation ===

Fold 1/5
Training data shape: (640, 228) (limited for CPU)
Validation data shape: (160, 228)
Training scikit-learn MLP model...
Iteration 1, loss = 0.64658149
Validation score: 0.750000
Iteration 2, loss = 0.44942465
Validation score: 0.750000
Iteration 3, loss = 0.38131087
Validation score: 0.718750
Iteration 4, loss = 0.33505745
Validation score: 0.750000
Iteration 5, loss = 0.30603417
Validation score: 0.750000
Iteration 6, loss = 0.27669732
Validation score: 0.750000
Iteration 7, loss = 0.26499178
Validation score: 0.734375
Validation score did not improve more than tol=0.000100 for 5 consecutive epochs. Stopping.
Training completed in 0.17 seconds

Fold 1 Results:
Fold: 1.0000
K: 5.0000
Test_Accuracy: 0.7500
Train_Accuracy: 0.7875
Precision: 0.6796
Recall: 0.9